In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_parquet("../input/ubiquant-parquet/train_low_mem.parquet")
print (len(df.index))

3141410


In [3]:
# for now, just use a sub-sample to do some exploration
#df = df.sample(frac=0.05, random_state=26)

# sample using investment_id
sampled_ids = pd.Series(df['investment_id'].unique()).sample(frac=0.35, random_state=26)
df = df.loc[df['investment_id'].isin(sampled_ids)]

In [4]:
features = ['f_'+str(i) for i in range(300)]

In [5]:
X = df[features].values
Y = df['target'].values

In [6]:
del df
gc.collect()

114

# Training

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
import lightgbm as lgb
import random

from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential, load_model, Model
from keras.layers import Input

In [8]:
pca = PCA(n_components=70, random_state=26)
X_pca = pca.fit_transform(X)

In [9]:
lr = LinearRegression()
lr.fit(X_pca, Y)

LinearRegression()

In [10]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'mse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 26
}
gb = lgb.train(params, lgb.Dataset(X, Y))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.626240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 1103323, number of used features: 300
[LightGBM] [Info] Start training from score -0.022359


In [11]:
def create_model_nn():
  model = Sequential([
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(256, activation='relu'),
      Dense(1)
  ])

  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

nn = create_model_nn()
nn.fit(X, Y, epochs=10, batch_size=1024)


User settings:

   KMP_AFFINITY=granularity=fine,noverbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER

Epoch 1/10
1078/1078 [==============================] - 51s 47ms/step - loss: 0.8593
Epoch 2/10
1078/1078 [==============================] - 50s 47ms/step - loss: 0.8349
Epoch 3/10
1078/1078 [==============================] - 52s 48ms/step - loss: 0.8313
Epoch 4/10
1078/1078 [==============================] - 50s 46ms/step - loss: 0.8286
Epoch 5/10
1078/1078 [==============================] - 50s 46ms/step - loss: 0.8268
Epoch 6/10
1078/1078 [==============================] - 49s 46ms/step - loss: 0.8256
Epoch 7/10
1078/1078 [==============================] - 50s 46ms/step - loss: 0.8244
Epoch 8/10
1078/1078 [==============================] - 49s 46ms/step - loss: 0.8235
Epoch 9/10
1078/1078 [==============================] - 50s 47ms/step - loss: 0.8225
Epoch 10/10
1078/1078 [==============================] - 61s 56ms/step - loss: 0.8215


# Submission

In [12]:
del X, Y
gc.collect()

836

In [13]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df['time_id'] = test_df['row_id'].apply(lambda x: int(x.split('_')[0]))
    X_test = test_df[features].values
    X_test_pca = pca.transform(X_test)
    preds_lr = lr.predict(X_test_pca)
    preds_gb = gb.predict(X_test)
    preds_nn = nn.predict(X_test)[:,0]
    preds = (preds_lr + preds_gb + preds_nn)/3.
    sample_prediction_df['target'] = preds
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
